### Defining the network, processing inputs, calling backward

In [1]:
# First network (MNIST CNN )
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())
print(params[2].size())
print(params[3].size())
print(params[4].size())
print(params[5].size())
print(params[6].size())
print(params[7].size())
print(params[8].size())
print(params[9].size())


10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [10]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0521, -0.0487, -0.0587, -0.0992,  0.1464,  0.0598, -0.0326, -0.0367,
         -0.0256,  0.2233]], grad_fn=<ThAddmmBackward>)


In [11]:
net.zero_grad()
print(out.backward(torch.randn(1, 10)))
print(input.grad)

None
None


torch.nn only accepts mini-batches, If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

### Loss Function

In [12]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()
# there are other losses

loss = criterion(output, target)
print(loss)

tensor(1.3851, grad_fn=<MseLossBackward>)


In [13]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

###  BackProp

In [14]:
net.zero_grad()     # zeroes the gradient buffers of all parameters (before starting sgd)

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0086,  0.0007,  0.0023,  0.0021, -0.0078,  0.0030])


###  Weight updates

In [15]:
# manually updating weights
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

    

In [18]:
params = list(net.parameters())
print(params)

[Parameter containing:
tensor([[[[-0.1862, -0.1496,  0.0201, -0.0144, -0.0385],
          [-0.0734, -0.1475, -0.1783,  0.0469, -0.1246],
          [-0.1170, -0.0707, -0.0914,  0.1348,  0.1972],
          [-0.1182, -0.0188,  0.0521, -0.0176, -0.1451],
          [-0.1851, -0.1346,  0.1965, -0.1993,  0.1852]]],


        [[[-0.1402, -0.0542,  0.0397,  0.0189, -0.0440],
          [ 0.0155,  0.0717,  0.1549,  0.0371, -0.1972],
          [ 0.1675,  0.1704,  0.0326,  0.1616, -0.0726],
          [-0.1405,  0.1913,  0.1534, -0.0872, -0.0068],
          [ 0.0080, -0.0805, -0.1816, -0.1202,  0.1645]]],


        [[[ 0.0261,  0.0236,  0.0641, -0.0029, -0.1618],
          [ 0.1385,  0.0227,  0.0386, -0.1392, -0.0843],
          [ 0.0571, -0.0162, -0.1808,  0.1128, -0.0065],
          [ 0.1614, -0.0830, -0.0743,  0.0074, -0.0097],
          [-0.0345, -0.1926, -0.0815,  0.0059,  0.1234]]],


        [[[-0.0305,  0.0817,  0.0022, -0.0534, -0.1564],
          [ 0.0065, -0.0198, -0.1114,  0.1549, -0.186

In [17]:
# using toch.optim
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update